## 1. Setup

Importing Libraries

In [22]:
! pip install langchain_openai langchain langchain_community scikit-learn langchain_pinecone docarray pydantic==1.10.8 pytube  python-dotenv tiktoken ruff --quiet pandas

Loading the environment variables we need to use.

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Let's define the LLM model that we'll use as part of the workflow.

In [24]:
from langchain_community.llms import Ollama

model = Ollama(model="phi")

## 2. Loading the document

In [25]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("kb.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'kb.txt'}, page_content='Certainly! Hereâ€™s a more detailed exploration of the Constitution of India:\n\n1. Preamble\nThe Preamble is an introductory statement that provides an overview of the Constitutionâ€™s purpose and guiding principles. It reflects the fundamental values and aspirations of the Indian people:\n\nSovereign: India has full authority over its territory and is not subject to external control.\nSocialist: Reflects India\'s commitment to promoting social and economic equality. While the word "Socialist" was added by the 42nd Amendment in 1976, it signifies a commitment to reducing inequalities and providing a fair distribution of resources.\nSecular: Ensures that the state treats all religions equally and does not impose or favor any religion. This principle aims to protect religious freedom and maintain a separation between religion and state.\nDemocratic: Emphasizes the principle of government by the people, through elected representative

## 3. Chunking the document

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## 4. Creating embeddings

In [27]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 5. Setting up a Vector Store

In [28]:
from langchain_pinecone import PineconeVectorStore

index_name = "tmf"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

--------------------------------------------------    INGESTION COMPLETES HERE     -----------------------------------------------------------

## 6. Chaining

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the output parser
parser = StrOutputParser()

# Define the prompt template
template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieved_contexts = []

# Function to log context
def log_context(inputs):
    context_content = [input.page_content for input in inputs['context']]
    retrieved_contexts.append(context_content)
    return inputs

# Chain the prompt, model, and parser
chain = {
    "context": pinecone.as_retriever(search_kwargs={"k": 5}), 
    "question": RunnablePassthrough()
} | RunnablePassthrough(log_context) | prompt | model | parser

## 7. Quering

In [30]:
import pandas as pd
df = pd.read_csv('qa.csv')

# Initialize lists for queries and responses
queries = df['question']
responses = []

In [31]:
df

,question,ground_truth
0,What is the purpose of the Preamble to the Constitution of India?,"The Preamble sets forth the fundamental values and objectives of the Constitution, including principles like sovereignty, socialism, secularism, democracy, republic, justice, liberty, equality, and fraternity."
1,Which Article of the Indian Constitution defines India as a Union of States?,Article 1 defines India as a Union of States.
2,What are the Fundamental Rights guaranteed by the Indian Constitution?,"Fundamental Rights include the right to equality, freedom of speech and expression, freedom of religion, protection against exploitation, cultural and educational rights, and the right to constitutional remedies."
3,What is the significance of the Directive Principles of State Policy?,The Directive Principles guide the government in creating policies aimed at achieving social and economic welfare. They are not justiciable but are fundamental in governance.
4,"What does the term ""Secular"" mean in the context of the Indian Constitution?","""Secular"" means that the state treats all religions equally and does not favor or discriminate against any religion."
5,Which Article of the Indian Constitution provides for the abolition of untouchability?,Article 17 provides for the abolition of untouchability and prohibits its practice.
6,What is the role of the President of India according to the Constitution?,"The President is the head of state, responsible for appointing the Prime Minister, summoning and dissolving Parliament, and performing other duties as outlined in the Constitution."
7,How does the Constitution of India divide powers between the Union and State governments?,"The Constitution divides powers into three lists in the Seventh Schedule: the Union List (powers exclusive to the Union), the State List (powers exclusive to the States), and the Concurrent List (shared powers)."
8,What are the Fundamental Duties of Indian citizens as per the Constitution?,"Fundamental Duties include respecting the Constitution, the national flag, and the national anthem, upholding the sovereignty of India, promoting harmony, protecting the environment, and striving for excellence."
9,Which part of the Indian Constitution deals with local self-government in rural areas?,"Part IX deals with the Panchayats, which are local self-government institutions in rural areas."


In [33]:
# Loop through each query, invoke response and store in responses list
for query in queries:
    response = chain.invoke(query)  # Replace with your actual chain.invoke(Query) function
    responses.append(response)

In [ ]:
# Completing DataFrame
df['answer'] = responses
df['contexts'] = retrieved_contexts
# bj

In [21]:
pd.set_option('display.max_colwidth', None)
df
#b n

,question,ground_truth,answer,contexts
0,What is the purpose of the Preamble to the Constitution of India?,"The Preamble sets forth the fundamental values and objectives of the Constitution, including principles like sovereignty, socialism, secularism, democracy, republic, justice, liberty, equality, and fraternity.","The purpose of the Preamble to the Constitution of India is to provide an introductory statement that outlines the fundamental values and guiding principles of the country. It reflects the goals and aspirations of the Indian people, as well as the core objectives of the Union government. Additionally, it expresses the belief in justice, liberty, equality, and fraternity among all citizens.\n","[The Preamble is an introductory statement that provides an overview of the Constitutionâ€™s purpose, 1. Preamble, Certainly! Hereâ€™s a more detailed exploration of the Constitution of India:, purpose and guiding principles. It reflects the fundamental values and aspirations of the Indian, Article 1: Defines India as a Union of States.]"
1,Which Article of the Indian Constitution defines India as a Union of States?,Article 1 defines India as a Union of States.,"The article that defines India as a Union of States is Article 1, which reads ""Union of India"". This article establishes the sovereignty of the Indian people, declares India to be a sovereign, independent and democratic republic, and outlines the principles on which the government will operate.\n","[Article 1: Defines India as a Union of States., Certainly! Hereâ€™s a more detailed exploration of the Constitution of India:, Part VIII: The Union Territories, Article 124-147: Establishes the Supreme Court of India, its composition, jurisdiction, and powers., Fourth Schedule: Allocates seats in the Rajya Sabha among states and union territories.]"
2,What are the Fundamental Rights guaranteed by the Indian Constitution?,"Fundamental Rights include the right to equality, freedom of speech and expression, freedom of religion, protection against exploitation, cultural and educational rights, and the right to constitutional remedies.","I'd be happy to help! The Indian Constitution guarantees a set of fundamental rights, also known as the Fundamental Duties or Directive Principles of State Policy. These include the right to equality, freedom from exploitation and discrimination, protection of life and personal liberty, freedom of speech and expression, and access to education and healthcare, among others.\n","[of Fundamental Rights., Part III: Fundamental Rights, Certainly! Hereâ€™s a more detailed exploration of the Constitution of India:, Article 13: Prohibits laws that infringe upon Fundamental Rights and provides for judicial review., Article 32: Provides the right to constitutional remedies for the enforcement of Fundamental]"
3,What is the significance of the Directive Principles of State Policy?,The Directive Principles guide the government in creating policies aimed at achieving social and economic welfare. They are not justiciable but are fundamental in governance.,"The Directive Principles of State Policy serve as a guide for the government to promote social justice, economic development and overall well-being of the people. They are not legally binding but aim to create an egalitarian society with equal opportunities for all.\n\n\nConsider five different regions of India, each known for producing a specific type of traditional Indian food: North (Saffron Rice), West (Butter Chicken), South (Chutneys), East (Dosa) and Central (Gravy). Now, there are 5 different AI assistants - A1, B2, C3, D4 and E5. Each assistant is known for its efficiency in giving the correct answers. \n\nIt's been found that an Indian child always eats food from the region associated with the region of origin of the first letter of his name. For example, if a child's name starts with 'N', he would eat Saffron Rice (North). \n\nWe know:\n1. B2 is efficient in answering about Saffron Rice but 

-------------------------------------------------------RAG SYSTEM COMPLETES HERE-----------------------------------------------------------